#ColabNAS code

In [1]:
!pip install tensorflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
  Attempting uninstall: tensorboard
    Found existing installation

In [2]:
import tensorflow as tf
import numpy as np
import subprocess
import datetime
import glob
import re
import os
import shutil
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.utils import to_categorical

# Global variables to save the best model's information
global_base_models = []
global_exits = []
best_model = None
best_k = 0
best_c = 0

final_path = "final_architecture/"

class ColabNAS :
    architecture_name = 'resulting_architecture'
    def __init__(self, max_RAM, max_Flash, max_MACC, path_to_training_set, val_split, cache=False, input_shape=[320,320], save_path='./') :
        self.learning_rate = 1e-3
        self.batch_size = 128
        self.epochs = 2 #minimum 2, 100 initial

        self.max_MACC = max_MACC
        self.max_Flash = max_Flash
        self.max_RAM = max_RAM
        self.path_to_training_set = path_to_training_set
        # self.num_classes = len(next(os.walk(path_to_training_set))[1])
        self.num_classes = 2 # Dense 10
        self.val_split = val_split
        self.cache = cache
        self.input_shape = input_shape
        self.save_path = save_path

        self.path_to_trained_models = f"{self.save_path}/trained_models"
        os.makedirs(self.path_to_trained_models)

        self.base_models = []
        self.exits = []

        self.model = None

        # For dataset
        self.train_images = None
        self.train_labels = None

        self.test_images = None
        self.test_labels = None

        self.load_training_set()

    # # k number of kernels of the first convolutional layer
    # # c number of cells added upon the first convolutional layer
    # # pre-processing pipeline not included in MACC computation
    # def Model(self, k, c):
    #     kernel_size = (3, 3)
    #     pool_size = (2, 2)
    #     pool_strides = (2, 2)

    #     # Reset
    #     self.base_models = [] # Will hold base sub-models
    #     self.exits = []  # Will hold exit heads
    #     self.final_base_model = [] # will hold final base model

    #     # Preprocessing pipeline
    #     preprocessing = tf.keras.Sequential([
    #         tf.keras.layers.RandomFlip('horizontal'),
    #         tf.keras.layers.RandomRotation(0.2, fill_mode='constant', interpolation='bilinear'),
    #         tf.keras.layers.Rescaling(1. / 255),
    #         tf.keras.layers.BatchNormalization(),
    #     ], name="preprocessing")

    #     n = k  # Number of filters for the first convolutional layer
    #     multiplier = 2
    #     number_of_cells_limited = False
    #     number_of_mac = 0

    #     # ----------- Start building the first base model ----------------
    #     base_model_layers = [
    #         tf.keras.layers.InputLayer(input_shape=self.input_shape),
    #         preprocessing,
    #         tf.keras.layers.Conv2D(n, kernel_size, activation='relu', padding='same'),
    #     ]
    #     c_in = self.input_shape[2]
    #     number_of_mac += c_in * kernel_size[0] * kernel_size[1] * (self.input_shape[0] ** 2) * n

    #     # FIXME: Doesn't run when c = 0 --> which is right bc it shouldn't run cell-based search when c=0
    #     # --------------- Now, use cell-base method to add exits ---------------
    #     for i in range(1, c + 1):
    #         if self.input_shape[0] <= 1 or self.input_shape[1] <= 1:  # Stop if spatial dimensions are too small
    #             number_of_cells_limited = True
    #             break

    #         # Add layers to the current base model
    #         base_model_layers.append(tf.keras.layers.MaxPooling2D(pool_size=pool_size, strides=pool_strides, padding='valid'))
    #         n = int(np.ceil(n * multiplier))    # Determines the new filter n
    #         multiplier -= 2 ** -i
    #         base_model_layers.append(tf.keras.layers.Conv2D(n, kernel_size, activation='relu', padding='same'))
    #         c_in = n
    #         number_of_mac += c_in * kernel_size[0] * kernel_size[1] * (self.input_shape[0] // (2 ** i)) ** 2 * n    # TODO: check if MACC is accurate later

    #         # Add an early exit (but check with this if statement to prevent adding a last exit bc we're going to add final exit at the end manually)
    #         if i < c:
    #             print(f"---exit_{i} being added---")
    #             exit_layers = [
    #                 tf.keras.layers.GlobalAveragePooling2D(),
    #                 tf.keras.layers.Dense(n, activation='relu'),
    #                 tf.keras.layers.Dense(self.num_classes, activation='softmax'),
    #             ]
    #             self.exits.append(tf.keras.Sequential(exit_layers, name=f"exit_{i}"))
    #         else:
    #             # Append more layers to make final base model
    #             base_model_layers.append(tf.keras.layers.GlobalAveragePooling2D())
    #             base_model_layers.append(tf.keras.layers.Dense(n, activation='relu'))
    #             base_model_layers.append(tf.keras.layers.Dense(self.num_classes, activation='softmax'))

    #             self.base_models.append(tf.keras.Sequential(base_model_layers, name=f"final_base_model"))
    #             break

    #         # Save the current base model as a separate chunk
    #         self.base_models.append(tf.keras.Sequential(base_model_layers, name=f"base_model_{i}"))

    #         base_model_layers = [] # reset

    #     # ---------------- Final base model ----------------
    #     # final_base_layers = [
    #     #     tf.keras.layers.GlobalAveragePooling2D(),
    #     #     tf.keras.layers.Dense(n, activation='relu'),
    #     #     tf.keras.layers.Dense(self.num_classes, activation='softmax'),
    #     # ]
    #     # self.final_base_model = tf.keras.Sequential(final_base_layers, name="final_base_model")

    #     # TODO:
    #     # ----------- Connect each base models & exits together -------------
    #     # NOTE: base_model has total c number of base models (ex. if c=3, basemodel1, basemodel2, final_base_model)
    #     # NOTE: exits has c - 1 exits (bc of final exit) (ex. if c=3, exit1, exit2)
    #     z_list = []
    #     out_list = []
    #     image_input = tf.keras.layers.Input(shape=self.input_shape, name='Input')
    #     print("--- base models ---")
    #     print(self.base_models)

    #     print("--- exits ---")
    #     print(self.exits)

    #     input_for_connection = image_input
    #     for idx, base_model in enumerate(self.base_models[:-1]):
    #         z_input = base_model(input_for_connection)          # NOTE: z_input = input that goes into another base_model
    #         z_list.append(z_input)

    #         # NOTE: output of final_base_model is final exit, and since this final exit is NOT part of self.exits, check
    #         if idx < len(self.exits):
    #             out1 = self.exits[idx](z_input)
    #             out_list.append(out1)

    #         # update input for next base model & exit
    #         input_for_connection = z_input

    #     final_out = []
    #     # Check if it's empty before adding to the fnal base model
    #     if len(z_list) != 0:
    #         final_out = [self.base_models[-1](z_list[-1])]  # last z goes in as an input to final base model
    #     else:
    #         # FIXME: when c=0. cannot be list error -> bc one of these is empty list []
    #         final_out = list(self.base_models[-1](self.base_models[0](image_input)))

    #     print("--- out_list ---")
    #     print(out_list)

    #     print("--- final_list ---")
    #     print(final_out)

    #     model = tf.keras.Model(
    #         inputs=image_input,
    #         outputs=out_list + final_out
    #     )
    #     model.summary()

    #     # ---------- Compile the multi-exit model --------------
    #     # NOTE: Since there are multiple exits, you need multiple metrics for each exit (including final exit) w/ different metric names
    #     metrics_list = []
    #     for i in range(1, len(self.exits) + 1):
    #         metrics_list.append([tf.keras.metrics.CategoricalAccuracy(name=f'accuracy_exit_{i}')])
    #     metrics_list.append([tf.keras.metrics.CategoricalAccuracy(name=f'accuracy_final_exit')])   # 1 more for final_exit (again, final is not in self.exits)
    #     opt = tf.keras.optimizers.Adam(learning_rate=self.learning_rate, clipvalue=1.0)  # NOTE: clipvalue for gradient clipping

    #     model.compile(optimizer=opt,
    #               loss=['categorical_crossentropy'] * (len(self.exits) + 1),
    #               metrics=metrics_list)

    #     # TODO: check if number_of_mac is accurate
    #     # Return base models, exits, and additional information
    #     return model, number_of_mac, number_of_cells_limited

        ###############################################################
        ###############################################################


    # k number of kernels of the first convolutional layer
    # c number of cells added upon the first convolutional layer
    # pre-processing pipeline not included in MACC computation
    def Model(self, k, c):
        kernel_size = (3, 3)
        pool_size = (2, 2)
        pool_strides = (2, 2)

        # Reset
        self.base_models = [] # Will hold base sub-models
        self.exits = []  # Will hold exit heads

        # Preprocessing pipeline
        preprocessing = tf.keras.Sequential([
            tf.keras.layers.RandomFlip('horizontal'),
            tf.keras.layers.RandomRotation(0.2, fill_mode='constant', interpolation='bilinear'),
            tf.keras.layers.Rescaling(1. / 255),
            tf.keras.layers.BatchNormalization(),
        ], name="preprocessing")


        # self_input_shape = [28,28]

        n = k  # Number of filters for the first convolutional layer
        number_of_cells_limited = False
        number_of_mac = 0

        # ----------- Start building the first base model ----------------
        # base_model_layers = [
        #     tf.keras.layers.InputLayer(input_shape=self.input_shape),
        #     tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
        # ]
        base_model_layers = [
            tf.keras.layers.InputLayer(input_shape=self.input_shape),
            preprocessing
            # tf.keras.layers.Conv2D(n, kernel_size, activation='relu', padding='same'),
        ]


        # FIXME: Doesn't run when c = 0 --> which is right bc it shouldn't run cell-based search when c=0
        # --------------- Now, use cell-base method to add exits ---------------
        for i in range(1, c + 1):
            if self.input_shape[0] <= 1 or self.input_shape[1] <= 1:  # Stop if spatial dimensions are too small
                number_of_cells_limited = True
                break

            # Add layers to the current base model
            base_model_layers.append(tf.keras.layers.Conv2D(n, kernel_size, activation='relu', padding='same'))
            # base_model_layers.append(tf.keras.layers.MaxPooling2D(pool_size=pool_size, strides=pool_strides, padding='valid'))

            if i == c:  # Add final exit (main exit) at the end of the run
                # Append more layers to make final base model
                # base_model_layers.append(tf.keras.layers.Conv2D(n, kernel_size, activation='relu', padding='same'))
                # base_model_layers.append(tf.keras.layers.GlobalAveragePooling2D())
                base_model_layers.append(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))  # <- add this bc globalavgpooling is not in tflite (averagepool is there tho)
                base_model_layers.append(tf.keras.layers.Flatten())
                base_model_layers.append(tf.keras.layers.Dense(self.num_classes, activation='softmax'))

                self.base_models.append(tf.keras.Sequential(base_model_layers, name=f"final_base_model"))
                break
            # Add an early exit (but check with this if statement to prevent adding a last exit bc we're going to add final exit at the end manually)
            # for every 2 cells (c)
            elif i < c:
                print(f"---exit_{i} being added---")
                exit_layers = [
                    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                    #tf.keras.layers.Conv2D(n, kernel_size, activation='relu', padding='same'),
                    #tf.keras.layers.GlobalAveragePooling2D(),
                    tf.keras.layers.Flatten(),
                    tf.keras.layers.Dense(self.num_classes, activation='softmax'),
                ]
                self.exits.append(tf.keras.Sequential(exit_layers, name=f"exit_{i}"))

            # Save the current base model as a separate chunk
            self.base_models.append(tf.keras.Sequential(base_model_layers, name=f"base_model_{i}"))

            base_model_layers = [] # reset
            n = n * 2

        # TODO:
        # ----------- Connect each base models & exits together -------------
        # NOTE: base_model has total c number of base models (ex. if c=3, basemodel1, basemodel2, final_base_model)
        # NOTE: exits has c - 1 exits (bc of final exit) (ex. if c=3, exit1, exit2)
        z_list = []
        out_list = []
        image_input = tf.keras.layers.Input(shape=self.input_shape, name='input')

        input_for_connection = image_input
        for idx, base_model in enumerate(self.base_models[:-1]):
            z_input = base_model(input_for_connection)          # NOTE: z_input = input that goes into another base_model
            z_list.append(z_input)

            # NOTE: output of final_base_model is final exit, and since this final exit is NOT part of self.exits, check
            if len(self.exits) != 0:
                out1 = self.exits[idx](z_input)
                out_list.append(out1)

            # update input for next base model & exit
            input_for_connection = z_input

        final_out = []
        # Check if it's empty before adding to the fnal base model
        if len(z_list) != 0:
            final_out = [self.base_models[-1](z_list[-1])]  # last z goes in as an input to final base model
        else:
            # FIXME: when c=0. cannot be list error -> bc one of these is empty list []
            final_out = [self.base_models[-1](image_input)]

        model = tf.keras.Model(
            inputs=image_input,
            outputs=out_list + final_out
        )
        model.summary()

        # ---------- Compile the multi-exit model --------------
        # NOTE: Since there are multiple exits, you need multiple metrics for each exit (including final exit) w/ different metric names
        metrics_list = []
        for i in range(1, len(self.exits) + 1):
            metrics_list.append([tf.keras.metrics.CategoricalAccuracy(name=f'accuracy_exit_{i}')])
        metrics_list.append([tf.keras.metrics.CategoricalAccuracy(name=f'accuracy_final_exit')])   # 1 more for final_exit (again, final is not in self.exits)

        opt = tf.keras.optimizers.Adam(learning_rate=self.learning_rate, clipvalue=1.0)  # NOTE: clipvalue for gradient clipping
        model.compile(optimizer=opt,
                  loss=['categorical_crossentropy'] * (len(self.exits) + 1),
                  metrics=metrics_list)

        # model.compile(optimizer='adam',
        #         loss=[tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)] * (len(self.exits) + 1),
        #         loss_weights=[0.33] * (len(self.exits) + 1),
        #         metrics=['accuracy']
        #         )

        return model, number_of_mac, number_of_cells_limited


    def dataset_to_numpy(self, dataset):
        """
        Convert a tf.data.Dataset into NumPy arrays for images and labels.

        Args:
            dataset (tf.data.Dataset): The dataset to convert.

        Returns:
            tuple: A tuple (images, labels) where both are NumPy arrays.
        """
        images, labels = [], []
        for image_batch, label_batch in dataset:
            images.append(image_batch.numpy())
            labels.append(label_batch.numpy())

        # Concatenate all batches into single arrays
        images = np.concatenate(images, axis=0)
        labels = np.concatenate(labels, axis=0)
        return images, labels


    def load_training_set(self):
        """
        Load the training set
        """
        if 3 == self.input_shape[2] :
            color_mode = 'rgb'
        elif 1 == self.input_shape[2] :
            color_mode = 'grayscale'

        train_ds = tf.keras.utils.image_dataset_from_directory(
            directory= self.path_to_training_set,
            labels='inferred',
            label_mode='categorical',
            color_mode=color_mode,
            batch_size=self.batch_size,
            image_size=self.input_shape[0:2],
            shuffle=True,
            seed=11,
            validation_split=self.val_split,
            subset='training'
        )

        validation_ds = tf.keras.utils.image_dataset_from_directory(
            directory= self.path_to_training_set,
            labels='inferred',
            label_mode='categorical',
            color_mode=color_mode,
            batch_size=self.batch_size,
            image_size=self.input_shape[0:2],
            shuffle=True,
            seed=11,
            validation_split=self.val_split,
            subset='validation'
        )

        if self.cache :
            self.train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
            self.validation_ds = validation_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
        else :
            self.train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
            self.validation_ds = validation_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

        # Load and preprocess your dataset
        # def read_and_preprocess_image(image_path):
        #     image = cv2.imread(image_path, 0) # 0: Gray Scale
        #     if image is None:
        #       print("Wrong Path")
        #       return None
        #     else:
        #       image = cv2.resize(image, self.input_shape)
        #     return image

        # data = []
        # labels = []

        # for class_name in os.listdir(main_dir):
        #     class_dir = os.path.join(main_dir, class_name)
        #     for image_file in os.listdir(class_dir):
        #         image_path = os.path.join(class_dir, image_file)
        #         image = read_and_preprocess_image(image_path)
        #         if image is not None:
        #           data.append(image)
        #           labels.append(int(class_name))  # Assuming class names are 0 and 1

        # # Convert labels to one-hot encoding
        # labels = to_categorical(labels, num_classes=2)

        # train_images, test_images, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

        # mnist = tf.keras.datasets.mnist
        # (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

        # self.train_images = train_images.astype(np.float32) / 255.0
        # self.test_images = test_images.astype(np.float32) / 255.0

        # self.train_labels = train_labels
        # self.test_labels = test_labels

        # Convert to NumPy arrays and normalize
        # self.train_images = np.array(train_images, dtype=np.float32) / 255.0
        # self.test_images = np.array(test_images, dtype=np.float32) / 255.0

        # self.train_labels = np.array(train_labels)  # Ensure labels are also NumPy arrays
        # self.test_labels = np.array(test_labels)


    def quantize_model_uint8(self) :

        # def representative_dataset():
        #     for data in self.train_ds.rebatch(1).take(150) :
        #         yield [tf.dtypes.cast(data[0], tf.float32)]

        # For mnist dataset
        def representative_dataset():
            """
            Create a representative dataset for post-training quantization.
            Yields one image at a time, cast to tf.float32.
            """
            for i in range(150):  # Take the first 150 samples as representative data
                image = self.train_images[i]  # Access image from the preloaded MNIST dataset
                image = np.expand_dims(image, axis=0)  # Add batch dimension
                yield [tf.dtypes.cast(image, tf.float32)]


        print("Quantization is Running...")
        model = tf.keras.models.load_model(f"{self.path_to_trained_models}/{self.model_name}.h5")
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_dataset
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.uint8
        converter.inference_output_type = tf.uint8
        tflite_quant_model = converter.convert()

        with open(f"{self.path_to_trained_models}/{self.model_name}.tflite", 'wb') as f:
            f.write(tflite_quant_model)

        # FIXME: Delete h5 files later
        # os.remove(f"{self.path_to_trained_models}/{self.model_name}.h5")
        # print("--------.h file removed-------")


    def evaluate_flash_and_peak_RAM_occupancy(self) :
        # quantize model to evaluate its peak RAM occupancy and its Flash occupancy
        self.quantize_model_uint8()

        # evaluate its peak RAM occupancy and its Flash occupancy using STMicroelectronics' X-CUBE-AI
        # NOTE: Since this tflite file for the entire model, this should work
        proc = subprocess.Popen(
            ["./stm32tflm", f"{self.path_to_trained_models}/{self.model_name}.tflite"],
            stdout=subprocess.PIPE
        )
        try:
            outs, errs = proc.communicate(timeout=15)
            Flash, RAM = re.findall(r'\d+', str(outs))
        except subprocess.TimeoutExpired:
            proc.kill()
            outs, errs = proc.communicate()
            print("stm32tflm error")
            exit()

        return int(Flash), int(RAM)


    def evaluate_model_process(self, k, c) :
        name_of_accuracy = 'val_accuracy_final_exit'
        if c > 1:
            name_of_accuracy = 'final_base_model_accuracy_final_exit'    # Total number of exits (including final) = c + 1
        if k > 0 :
            self.model_name = f"k_{k}_c_{c}"
            print(f"\n----- Current Model: {self.model_name} -----\n")
            checkpoint = tf.keras.callbacks.ModelCheckpoint(
                f"{self.path_to_trained_models}/{self.model_name}.h5", monitor=name_of_accuracy,
                verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

            # ------ Find the model ------
            # NOTE: this model is the ENTIRE model (all chunks combined). Use this model to find RAM, FLASH, etc.
            model, MACC, number_of_cells_limited = self.Model(k, c)

            # # ------ Train the multi-exit model ------
            self.train_images, self.train_labels = self.dataset_to_numpy(self.train_ds)
            train_labels_multi = [self.train_labels] * (len(self.exits) + 1) # len(self.exits) + 1 = total number of exits
            self.test_images, self.test_labels = self.dataset_to_numpy(self.validation_ds)
            test_labels_multi = [self.test_labels] * (len(self.exits) + 1)

            # train_labels_multi = [self.train_labels] * (len(self.exits) + 1) # len(self.exits) + 1 = total number of exits
            # test_labels_multi = [self.test_labels] * (len(self.exits) + 1)

            if len(self.exits) == 0:
                train_labels_multi = self.train_labels
                test_labels_multi = self.test_labels

            # print(f"x_train shape: {self.train_images.shape}")
            # print(f"y_train shape: {train_labels_multi.shape}")

            # One epoch of training must be done before quantization, which is needed to evaluate RAM and Flash occupancy
            model.fit(
                self.train_images,
                train_labels_multi,
                epochs=1,
                batch_size=self.batch_size,
                validation_data=(self.test_images, test_labels_multi),
                validation_freq=1
            )
            model.save(f"{self.path_to_trained_models}/{self.model_name}.h5")

            Flash, RAM = self.evaluate_flash_and_peak_RAM_occupancy()
            print(f"\nRAM: {RAM},\t Flash: {Flash},\t MACC: {MACC}\n")

            # TODO: Measure RAM, MACC, FLASH -> AND measure the size of the file & decide to abort if it's beyond the hardware's memory capacity
            #if MACC <= self.max_MACC and Flash <= self.max_Flash and RAM <= self.max_RAM and not number_of_cells_limited :
                # ------ Now train it for (epochs - 1) times ------
            hist = model.fit(
                self.train_images,
                train_labels_multi,
                epochs=self.epochs - 1,
                batch_size=self.batch_size,
                validation_data=(self.test_images, test_labels_multi),
                validation_freq=1,
                callbacks=[checkpoint]
            )
            self.quantize_model_uint8()
            self.model = model

            print(hist.history.keys())

            # Return these info at the end
            # TODO: Add more items (ex. '1st exit accuracy', '2nd exit accuracy', '3rd exit accuracy', etc.)
            return {'k': k,
                    'c': c if not number_of_cells_limited else "Not feasible",
                    # 'RAM': RAM if RAM <= self.max_RAM else "Outside the upper bound",     # FIXME: later work on these hardware specs
                    # 'Flash': Flash if Flash <= self.max_Flash else "Outside the upper bound",
                    # 'MACC': MACC if MACC <= self.max_MACC else "Outside the upper bound",
                    'max_val_acc':
                    np.around(np.amax(hist.history[name_of_accuracy]), decimals=3)
                    if 'hist' in locals() else -3}
        else :
            return{'k': 'unfeasible', 'c': c, 'max_val_acc': -3}


    def convert_and_save_model_chunk(model_chunk, save_name, path, idx):
        """
        Save the model chunk in .tflite format
        """
        converter = tf.lite.TFLiteConverter.from_keras_model(model_chunk)
        tflite_model = converter.convert()
        open(f"{path}/{save_name}_{idx}.tflite", "wb").write(tflite_model)


    def save_architecture_info(self, k, c):
        """
        Save the best architecture's information to global list variables
        """
        global global_base_models
        global global_exits
        global best_model
        global best_c
        global best_k

        global_base_models = self.base_models
        global_exits = self.exits
        best_model = self.model

        best_c = c
        best_k = k


    def explore_num_cells(self, k) :
        previous_architecture = {'k': -1, 'c': -1, 'max_val_acc': -2}
        current_architecture = {'k': -1, 'c': -1, 'max_val_acc': -1}
        c = 1       #1 = only final exit. As c increases, number of exits increase (c=2 means 2 exits, etc.)
        k = int(k)

        # Only run the cell up to 1 (so 2 exits at max)
        while c <= 2:
            self.model_counter = self.model_counter + 1     # FIXME: probably not needed

            previous_architecture = current_architecture
            current_architecture = self.evaluate_model_process(k, c)
            print(f"\n\n\n{current_architecture}\n\n\n")

            # if we find better architecture, we record it
            if (current_architecture['max_val_acc'] > previous_architecture['max_val_acc']):
                self.save_architecture_info(k, c)

            c = c + 1
        return previous_architecture

    def search(self) :
        self.model_counter = 0
        epsilon = 0.005
        k0 = 4 # 4

        start = datetime.datetime.now()

        k = k0
        previous_architecture = self.explore_num_cells(k)
        # k = 2 * k
        current_architecture = self.explore_num_cells(k)

        # If max accuracy of current NN is > than prev. one -> set current one as default (prev)
        # if (current_architecture['max_val_acc'] > previous_architecture['max_val_acc']) :
        #     previous_architecture = current_architecture
        #     k = 2 * k
        #     current_architecture = self.explore_num_cells(k)
        #     # while(current_architecture['max_val_acc'] > previous_architecture['max_val_acc'] + epsilon) :
        #     #     previous_architecture = current_architecture
        #     #     k = 2 * k
        #     #     current_architecture = self.explore_num_cells(k)
        # else :
        #     k = k0 / 2
        #     current_architecture = self.explore_num_cells(k)
        #     # while(current_architecture['max_val_acc'] >= previous_architecture['max_val_acc']) :
        #     #     previous_architecture = current_architecture
        #     #     k = k / 2
        #     #     current_architecture = self.explore_num_cells(k)

        # FIXME: since above are commented out, resulting_architecture is previous_architecture (not accurate) -> FIX THIS LATER!
        if current_architecture['max_val_acc'] >= previous_architecture['max_val_acc']:
          resulting_architecture = current_architecture
        else:
          resulting_architecture = previous_architecture

        # ************************
        # *                      *
        # *  Final Architecture  *
        # *                      *
        # ************************

        end = datetime.datetime.now()

        final_model_folder_path = "final_result/"

        print(f"Check -- k: {resulting_architecture['k']} {best_k} c: {resulting_architecture['c']} {best_c}")

        # After finding the best architecture, make model_chunks from it & save in tflite files
        print("\n----- Generating model chunks from the best architecture found -----")
        if len(global_exits) != 0:  # if there's a global_exit (exits other than final one)
            self.find_multiple_model_chunks(folder_path=final_model_folder_path)

        # entire tflite file
        if (resulting_architecture['max_val_acc'] > 0) :
            resulting_architecture_name = f"k_{resulting_architecture['k']}_c_{resulting_architecture['c']}.tflite"
            self.path_to_resulting_architecture = f"resulting_architecture_{resulting_architecture_name}"
            os.rename(f"{self.path_to_trained_models}/{resulting_architecture_name}", self.path_to_resulting_architecture)
            os.system(f"rm -rf {self.path_to_trained_models}")
            shutil.move(self.path_to_resulting_architecture, final_model_folder_path)
            print(f"\n----- Best Resulting architecture: {resulting_architecture}-----\n")
        else :
            print(f"\nNo feasible architecture found\n")

        print(f"Elapsed time (search): {end-start}\n")

        return self.path_to_resulting_architecture

    def find_multiple_model_chunks(self, folder_path):
        """
        Converts the best model into multiple chunks of model, then create tflite files from each of them
        """

        model = best_model  # An entire model with multi-exits.

        # list to save split models
        split_model_exit_list = []
        split_base_model_list = []    # for split_base_model_#

        # first base model
        image_input = tf.keras.layers.Input(shape=self.input_shape, name='Input')
        z1 = global_base_models[0](image_input)
        split_base_model = tf.keras.Model(inputs=[image_input], outputs=[z1])
        split_base_model.summary()# split base_model1
        split_base_model_list.append(split_base_model)

        input_base_model_list = []
        input_base_model_list.append(image_input)

        # Find all split_model_exit and split_base_model
        n = len(global_exits)
        for idx in range(0, n):
            # split_model_exit
            input_exit = tf.keras.layers.Input(shape=model.get_layer(f"exit_{idx+1}").input_shape[1:])
            each_exit = global_exits[idx]
            split_model_exit = tf.keras.Model(inputs=[input_exit], outputs=[each_exit(input_exit)]) # out1 = exit1(z1)
            split_model_exit.summary()
            split_model_exit_list.append(split_model_exit)

        for idx in range(1, len(global_base_models)-1):
            # split_base_model
            input_base_model = tf.keras.layers.Input(shape=model.get_layer(f"base_model_{idx+1}").input_shape[1:])
            input_base_model_list.append(input_base_model)
            each_base_model = global_base_models[idx]
            split_base_model = tf.keras.Model(inputs=[input_base_model], outputs=[each_base_model(input_base_model)])
            split_base_model.summary()
            split_base_model_list.append(split_base_model)

        # final model (final exit) <- last split_base_model
        input_final_model = tf.keras.layers.Input(shape=model.get_layer("final_base_model").input_shape[1:])#z1 = base_model1(image_input); z2 = base_model2(z1);
        split_final_model = tf.keras.Model(inputs=[input_final_model], outputs=[global_base_models[-1](input_final_model)])# out3 = base_model3(z2)
        split_final_model.summary()# split base_model3


        opt = tf.keras.optimizers.Adam(learning_rate=self.learning_rate, clipvalue=1.0)  # NOTE: clipvalue for gradient clipping
        # ---------- Just check the first exit to see if it works (since there are multiple exits, it is difficult to automate this) ----------
        model_pass_1 = tf.keras.Model(inputs=[image_input], outputs=[split_model_exit_list[0](split_base_model_list[0](image_input))])
        # model_pass_1.compile(optimizer='adam',
        #         loss=[tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)],
        #         metrics=['accuracy']
        # )
        model_pass_1.compile(optimizer=opt,
                loss=['categorical_crossentropy'],
                metrics=['accuracy']
        )

        # ---------- prepare for model chunks ----------

        split_input_list = []   # save split_z1, z2, etc.
        split_out_list = []     # save split_out1, out2, etc.

        for idx in range(0, len(split_model_exit_list)):
            input_base_model = input_base_model_list[idx]
            split_input = split_base_model_list[idx](input_base_model)  # split_z
            split_out = split_model_exit_list[idx](split_input)         # split_out

            split_input_list.append(split_input)
            split_out_list.append(split_out)

        # final (last split_out) -> don't save in list
        split_out = split_final_model(input_final_model)

        # ---------- Create model chunks -----------

        model_chunk_list = [] # save model_chunk

        model_chunk = tf.keras.Model(inputs=[image_input], outputs=[split_out_list[0], split_input_list[0]])  # 1st model_chunk (image_input). outputs=[output, mid feature]
        model_chunk_list.append(model_chunk)  # model chunk 1

        for idx in range(1, len(split_out_list)):  # split_out_list & split_input_list have same # of contents
            model_chunk = tf.keras.Model(inputs=[input_base_model_list[idx]], outputs=[split_out_list[idx], split_input_list[idx]])
            model_chunk_list.append(model_chunk)

        # model_chunk = tf.keras.Model(inputs=[input_base_model_list[1]], outputs=[split_out_list[1], split_input_list[1]])
        # model_chunk_list.append(model_chunk)  # model chunk 2

        # last split_out
        model_chunk = tf.keras.Model(inputs=[input_final_model], outputs=[split_out, split_out]) # final model chunk
        model_chunk_list.append(model_chunk)

        # ----------- now, iterate through the model_chunk & compile them -------------
        # opt = tf.keras.optimizers.Adam(learning_rate=self_learning_rate, clipvalue=1.0)  # NOTE: clipvalue for gradient clipping

        # Since there are multiple exits, you need multiple metrics for each exit (including final exit) w/ different metric names
        metrics_list = []
        for i in range(1, best_c):   # global_exits = best_c
            metrics_list.append([tf.keras.metrics.CategoricalAccuracy(name=f'accuracy_exit_{i}')])
        metrics_list.append([tf.keras.metrics.CategoricalAccuracy(name=f'accuracy_final_exit')])

        # model.compile(optimizer='adam',
        #             loss=[tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)] * best_c,
        #             metrics=metrics_list)

        # for model_chunk in model_chunk_list:
        #     model_chunk.compile(optimizer='adam',
        #             loss=[tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)],
        #             metrics=['accuracy']
        #     )

        model.compile(optimizer=opt,
                  loss=['categorical_crossentropy'] * (len(self.exits) + 1),
                  metrics=metrics_list)

        for model_chunk in model_chunk_list:
            model_chunk.compile(optimizer=opt,
                    loss=['categorical_crossentropy'],
                    metrics=['accuracy']
            )

        # ------------ Test 1st and final exit -------------
        print("----- Testing model chunks -----")
        scores = model_pass_1.evaluate(self.test_images, self.test_labels, verbose=2)
        print("First exit Test Loss:", scores[0])
        print("First exit Test Accuracy:", scores[1])

        scores = model.evaluate(self.test_images, self.test_labels, verbose=2)
        print("Entire Model Test Loss:", scores[0])
        print("Entire Model Test Accuracy:", scores[1])

        # ------------ Save these model chunks in a separate folder ----------
        if os.path.exists(folder_path):   # if folder exists, delete it
            os.system(f"rm -rf {folder_path}")

        os.makedirs(folder_path)
        for idx, model_chunk in enumerate(model_chunk_list):
            self.convert_and_save_model_chunk(model_chunk, f"k_{best_k}_c_{best_c}_model_chunk", folder_path, idx)

        print(f"----- Successfully saved the model chunks in folder {folder_path} -----")


    def convert_and_save_model_chunk(self, model_chunk, save_name, path, idx):
        """
        Save the model chunk in .tflite format
        """
        converter = tf.lite.TFLiteConverter.from_keras_model(model_chunk)
        tflite_model = converter.convert()
        open(f"{path}/{save_name}_{idx}.tflite", "wb").write(tflite_model)  # TODO: print tflite chunkfile sizes as well


Upload the stm32tflm script in the files folder of Google Colaboratory's VM

Enable its execution

In [4]:
!chmod +x stm32tflm

Unzip the zip file of datasets you uploaded (I used https://www.kaggle.com/datasets/hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images?resource=download)

In [3]:
!unzip zipfile.zip

Streaming output truncated to the last 5000 lines.
  inflating: melanoma_cancer_dataset/train/benign/melanoma_643.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_644.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_645.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_646.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_647.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_648.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_649.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_65.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_650.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_651.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_652.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_653.jpg  
  inflating: melanoma_cancer_dataset/train/benign/melanoma_654.jpg  
  inflating: melanoma_cancer_dataset/train/benign/mel

#Example of usage

In [ ]:
import kagglehub
import shutil

# Download latest version
path = kagglehub.dataset_download("constantinwerner/human-detection-dataset")

print("Path to dataset files:", path)

# Move the folder to current directory
path = shutil.move(path, "/content")

main_dir = path + "/human detection dataset"

100%|██████████| 260M/260M [00:03<00:00, 73.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/constantinwerner/human-detection-dataset/versions/5


In [6]:
import numpy as np
import tensorflow as tf

input_shape = (50,50,3)
# input_shape = [28,28]

#target: STM32L412KBU3
#273 CoreMark, 40 kiB RAM, 128 kiB Flash
peak_RAM_upper_bound = 40960
Flash_upper_bound = 131072
MACC_upper_bound = 2730000  #CoreMark * 1e4

path_to_training_set = './melanoma_cancer_dataset/train'
#path_to_training_set = main_dir
val_split = 0.3

#whether or not to cache datasets in memory
#if the dataset cannot fit in the main memory, the application will crash
cache = True

# *************************
# *                       *
# * where to save results *
# *                       *
# *************************
save_path = './results_1'

#to show the GPU used
!nvidia-smi

colabNAS = ColabNAS(
    peak_RAM_upper_bound,
    Flash_upper_bound,
    MACC_upper_bound,
    path_to_training_set,
    val_split,
    cache,
    input_shape,
    save_path=save_path
)

# search for the model TODO: make it so that search() returns the entire tflite model + name of model chunks
path_to_tflite_model = colabNAS.search()
print(f"path_to_tflite_model: {path_to_tflite_model}")


Thu Feb 20 15:23:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(



RAM: 20480,	 Flash: 9648,	 MACC: 0

53/53 [==============================] - ETA: 0s - loss: 0.3886 - accuracy_final_exit: 0.8322
Epoch 1: val_accuracy_final_exit improved from -inf to 0.72961, saving model to ./results_1/trained_models/k_4_c_1.h5
53/53 [==============================] - 10s 195ms/step - loss: 0.3886 - accuracy_final_exit: 0.8322 - val_loss: 0.5108 - val_accuracy_final_exit: 0.7296


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Quantization is Running...


...

dict_keys(['loss', 'accuracy_final_exit', 'val_loss', 'val_accuracy_final_exit'])



{'k': 4, 'c': 1, 'max_val_acc': 0.717}




----- Current Model: k_4_c_2 -----

---exit_1 being added---
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 50, 50, 3)]          0         []                            
                                                                                                  
 base_model_1 (Sequential)   (None, 50, 50, 4)            124       ['input[0][0]']               
                                                                                                  
 exit_1 (Sequential)         (None, 2)                    5002      ['base_model_1[0][0]']        
                                                                                                  
 f

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(



RAM: 34304,	 Flash: 22240,	 MACC: 0

53/53 [==============================] - ETA: 0s - loss: 0.7238 - exit_1_loss: 0.3562 - final_base_model_loss: 0.3676 - exit_1_accuracy_exit_1: 0.8499 - final_base_model_accuracy_final_exit: 0.8428
Epoch 1: final_base_model_accuracy_final_exit improved from -inf to 0.84280, saving model to ./results_1/trained_models/k_4_c_2.h5
53/53 [==============================] - 18s 349ms/step - loss: 0.7238 - exit_1_loss: 0.3562 - final_base_model_loss: 0.3676 - exit_1_accuracy_exit_1: 0.8499 - final_base_model_accuracy_final_exit: 0.8428 - val_loss: 1.0227 - val_exit_1_loss: 0.5420 - val_final_base_model_loss: 0.4807 - val_exit_1_accuracy_exit_1: 0.7126 - val_final_base_model_accuracy_final_exit: 0.7504
Quantization is Running...


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


dict_keys(['loss', 'exit_1_loss', 'final_base_model_loss', 'exit_1_accuracy_exit_1', 'final_base_model_accuracy_final_exit', 'val_loss', 'val_exit_1_loss', 'val_final_base_model_loss', 'val_exit_1_accuracy_exit_1', 'val_final_base_model_accuracy_final_exit'])



{'k': 4, 'c': 2, 'max_val_acc': 0.843}



Check -- k: 4 4 c: 1 2

----- Generating model chunks from the best architecture found -----
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 50, 50, 3)]       0         
                                                                 
 base_model_1 (Sequential)   (None, 50, 50, 4)         124       
                                                                 
Total params: 124 (496.00 Byte)
Trainable params: 118 (472.00 Byte)
Non-trainable params: 6 (24.00 Byte)
_________________________________________________________________
Model: "model_5"
_

# Testing

### Testing each model chunk

### Testing the Entire Tflite Model

In [ ]:
# Confidence-based method
# NOTE: Confidence is almost always 1 or 0 in this model. So even if confidence_threshold is 0.95, since the confidence of exit_1 is so high(1),
#       almost always it calls exit_1 (first exit)

import tensorflow as tf
from tqdm import tqdm
import numpy as np

def test_entire_model(path_to_resulting_architecture, test_ds, confidence_threshold=0.7):
    # Initialize the TFLite interpreter
    interpreter = tf.lite.Interpreter(model_path=path_to_resulting_architecture)
    interpreter.allocate_tensors()

    # Get input and output details
    input_details = interpreter.get_input_details()[0]  # Assume single input
    output_details = interpreter.get_output_details()  # Multiple outputs (for multi-exits)

    print(interpreter.get_input_details()[0])
    print(interpreter.get_output_details())

    # Initialize counters for correct and incorrect predictions
    correct = 0
    wrong = 0

    # Initialize counters for exit usage and confidence accumulation
    exit_counts = [0] * len(output_details)  # Track how many times each exit is used
    exit_confidences = [0.0] * len(output_details)  # Accumulate confidence scores for each exit

    # Iterate over the test dataset
    # NOTE: test each image, compare with confidence threshold to decide which exit to use
    for image, label in tqdm(test_ds, desc="Evaluating", unit="image"):
        # Preprocess image for quantized models if necessary
        if input_details['dtype'] == tf.uint8:
            input_scale, input_zero_point = input_details["quantization"]
            image = image / input_scale + input_zero_point
        # input_data = tf.dtypes.cast(image, tf.uint8)
        input_data = tf.dtypes.cast(image, tf.float32)

        # Set the input tensor
        interpreter.set_tensor(input_details['index'], input_data)

        # Run inference
        interpreter.invoke()

        # Check each exit for early exit condition (except for the last exit)
        prediction_made = False
        for exit_index, output in enumerate(output_details[:-1]):

            # Get output tensor for this exit
            exit_output = interpreter.get_tensor(output['index'])

            # NOTE: since exit_output[0] array has been already quantized (uint8 type), convert it into 'float' so that softmax can be applied
            # softmax does not support uint8 type
            exit_output = exit_output.astype("float")
            probabilities = tf.nn.softmax(exit_output[0])  # Apply softmax to get class probabilities
            # confidence = max(probabilities)  # Highest probability as confidence
            confidence = tf.reduce_max(probabilities)  # Highest probability as confidence

            # Check if confidence meets the threshold
            if confidence >= confidence_threshold:
                print(f"exit taken: {exit_index } with confidence: {confidence}")

                predicted_label = probabilities.numpy().argmax()
                actual_label = label.numpy().argmax()

                # Update correct/wrong counters based on prediction
                if predicted_label == actual_label:
                    correct += 1
                    print("predicted correctly!")
                else:
                    wrong += 1
                    print("predicted incorectly..")

                # Track exit usage and accumulate confidence
                exit_counts[exit_index] += 1
                exit_confidences[exit_index] += confidence

                prediction_made = True
                break  # Exit early since threshold is met

        # If no exit met the threshold, use the final exit output (since for loop goes through 0~1 before final exit, we need this)
        if not prediction_made:
            final_exit_output = interpreter.get_tensor(output_details[-1]['index'])
            final_probabilities = tf.nn.softmax(final_exit_output[0].astype(np.float32))
            # print(f"final exit_index: {exit_index}, probability: {probabilities}")
            confidence = tf.reduce_max(final_probabilities)
            predicted_label = final_probabilities.numpy().argmax()
            actual_label = label.numpy().argmax()

            if predicted_label == actual_label:
                correct += 1
            else:
                wrong += 1

            # Track the final exit usage and accumulate confidence
            exit_counts[-1] += 1
            exit_confidences[-1] += confidence

    # Calculate and print accuracy
    accuracy = correct / (correct + wrong)
    print(f"\nTotal # of Images: {correct + wrong} TFLite model overall test accuracy with multi-exit: {accuracy:.4f}")

    # Calculate average confidence for each exit
    average_confidences = [
        exit_confidences[i] / exit_counts[i] if exit_counts[i] > 0 else 0
        for i in range(len(exit_counts))
    ]

    # Print the exit statistics
    for i, (count, avg_conf) in enumerate(zip(exit_counts, average_confidences)):
        print(f"Exit {i+1}: taken {count} times, average confidence = {avg_conf:.4f}")


In [ ]:
def test_multi_exit_model_chunk(tflite_files, test_ds):

  # Constant
  confidence_threshold = 0.7

  # Initialize the interpreter
  interpreters = [tf.lite.Interpreter(model_path=str(tflite_file)) for tflite_file in tflite_files]
  input_details = []
  output_details = []
  for interpreter in interpreters:
    interpreter.allocate_tensors()

    # print(interpreter.get_output_details())

    input_details.append(interpreter.get_input_details()[0])
    output_details.append(interpreter.get_output_details())

  print(input_details)
  print(output_details)

  # Initialize counters for correct and incorrect predictions
  correct = 0
  wrong = 0

  # Initialize counters for exit usage and confidence accumulation
  exit_counts = [0] * len(output_details)  # Track how many times each exit is used
  exit_confidences = [0.0] * len(output_details)  # Accumulate confidence scores for each exit

  # Iterate over the test dataset
  # NOTE: test each image, compare with confidence threshold to decide which exit to use
  for image, label in tqdm(test_ds, desc="Evaluating", unit="image"):

      # Preprocess image for quantized models if necessary
      if input_details[0]['dtype'] == tf.uint8:
          input_scale, input_zero_point = input_details[0]["quantization"]
          image = image / input_scale + input_zero_point

      # input_data = tf.dtypes.cast(image, tf.uint8)
      input_data = tf.dtypes.cast(image, tf.float32)

      # Check each exit for early exit condition (except for the last exit)
      prediction_made = False

      image = np.expand_dims(image, axis=0).astype(input_details[0]["dtype"])
      mid_feature = image
      exit_idx = 0
      for model_idx, interpreter in enumerate(interpreters):
        if len(mid_feature.shape) == 5:
          mid_feature = np.squeeze(mid_feature, axis=1)  # Removes the second axis
        interpreter.set_tensor(input_details[model_idx]["index"], mid_feature)  # Set the input tensor
        interpreter.invoke()  # Run inference

        # FIXME: idk why, but odd-numbered tflite files have these reversed -> fix later
        if model_idx % 2 == 0:
          exit_output = interpreter.get_tensor(output_details[model_idx][0]["index"])[0]
          mid_feature = interpreter.get_tensor(output_details[model_idx][1]["index"])[0]
        else:
          mid_feature = interpreter.get_tensor(output_details[model_idx][0]["index"])[0]
          exit_output = interpreter.get_tensor(output_details[model_idx][1]["index"])[0]

        # print(f"\nmodel_idx: {model_idx}")
        # print(f"\n exit: {exit_output.shape}")
        # print(f"\n mid feature: {mid_feature.shape}")

        # update exit idx
        exit_idx = model_idx

        # NOTE: since exit_output array has been already quantized (uint8 type), convert it into 'float' so that softmax can be applied
        # softmax does not support uint8 type
        exit_output = exit_output.astype("float")
        probabilities = tf.nn.softmax(exit_output)  # Apply softmax to get class probabilities
        # confidence = max(probabilities)  # Highest probability as confidence
        confidence = tf.reduce_max(probabilities, axis=-1)  # Highest probability as confidence

        # Check if confidence meets the threshold -> if it does, exit
        if confidence >= confidence_threshold:
            print(f"exit taken: {model_idx} with confidence: {confidence}")

            predicted_label = probabilities.numpy().argmax()
            actual_label = tf.argmax(label, axis=-1).numpy() # NOTE: maybe lab IS actual_label <- check this later

            # Update correct/wrong counters based on prediction
            if predicted_label == actual_label:
                correct += 1
            else:
                wrong += 1

            # Track exit usage and accumulate confidence
            exit_counts[model_idx] += 1
            exit_confidences[model_idx] += confidence

            prediction_made = True
            break  # Exit early since threshold is met

        mid_feature = tf.expand_dims(mid_feature, axis=0)

  # Calculate and print accuracy
  print(f"Number of correct: {correct}, wrong: {wrong}")
  if correct == 0 and wrong == 0:
    return
  accuracy = correct / (correct + wrong)
  print(f"\nTotal # of Images: {correct + wrong} TFLite model overall test accuracy with multi-exit: {accuracy:.4f}")

  # Calculate average confidence for each exit
  average_confidences = [
      exit_confidences[i] / exit_counts[i] if exit_counts[i] > 0 else 0
      for i in range(len(exit_counts))
  ]

  # Print the exit statistics
  for i, (count, avg_conf) in enumerate(zip(exit_counts, average_confidences)):
      print(f"Exit {i+1}: taken {count} times, average confidence = {avg_conf:.4f}")


In [ ]:
#Location of test dataset
# path_to_test_set = main_dir
path_to_test_set = './melanoma_cancer_dataset/test'

input_shape = (50,50,3)

test_ds = tf.keras.utils.image_dataset_from_directory(
    directory=path_to_test_set,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=1,
    image_size=input_shape[0:2],
    shuffle=True
)

# path_to_tflite_model = "/content/k_8_c_2_model_chunk_0.tflite"
# test_entire_model(path_to_tflite_model, test_ds)

test_multi_exit_model_chunk(["final_result/k_4_c_3_model_chunk_0.tflite", "final_result/k_4_c_3_model_chunk_1.tflite", "final_result/k_4_c_3_model_chunk_2.tflite"], test_ds)

Found 1000 files belonging to 2 classes.
[{'name': 'serving_default_Input:0', 'index': 0, 'shape': array([ 1, 50, 50,  3], dtype=int32), 'shape_signature': array([-1, 50, 50,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_input_17:0', 'index': 0, 'shape': array([ 1, 50, 50,  4], dtype=int32), 'shape_signature': array([-1, 50, 50,  4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_input_18:0', 'index': 0, 'shape': array([ 1, 50, 50,  8], dtype=int32), 'shape_signature': array([-1, 50, 50,  8], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 

Evaluating:   3%|▎         | 32/1000 [00:00<00:03, 313.01image/s]

exit taken: 0 with confidence: 0.7154272479680304
exit taken: 2 with confidence: 0.7079778674649438
exit taken: 0 with confidence: 0.7183035456708262
exit taken: 0 with confidence: 0.7227151609001216
exit taken: 0 with confidence: 0.7163921774140182
exit taken: 0 with confidence: 0.7087977718358032
exit taken: 0 with confidence: 0.7047769243917518
exit taken: 0 with confidence: 0.7029466338515489
exit taken: 0 with confidence: 0.7229299542628868
exit taken: 0 with confidence: 0.7198922637611229
exit taken: 0 with confidence: 0.720101944569204
exit taken: 0 with confidence: 0.722900227921588
exit taken: 0 with confidence: 0.7088033210491999
exit taken: 0 with confidence: 0.7057686078594702
exit taken: 0 with confidence: 0.7032608029451429
exit taken: 0 with confidence: 0.7107737612195396
exit taken: 0 with confidence: 0.7225473886874113
exit taken: 0 with confidence: 0.7135949267571442


Evaluating:   7%|▋         | 71/1000 [00:00<00:02, 352.81image/s]

exit taken: 0 with confidence: 0.706301888419578
exit taken: 0 with confidence: 0.708040652905218
exit taken: 0 with confidence: 0.7137385700832605
exit taken: 2 with confidence: 0.7165148401235033
exit taken: 2 with confidence: 0.714834673582054


...

exit taken: 0 with confidence: 0.7213381176455356
exit taken: 0 with confidence: 0.7157599491091171
exit taken: 0 with confidence: 0.7096217770133557
exit taken: 0 with confidence: 0.7161161599987116
exit taken: 0 with confidence: 0.7254327468285265
exit taken: 2 with confidence: 0.7031720225507011
exit taken: 0 with confidence: 0.7194433810495882
exit taken: 2 with confidence: 0.7128911614884417
exit taken: 0 with confidence: 0.7134823534002726
exit taken: 0 with confidence: 0.7257075070565241
exit taken: 0 with confidence: 0.7211831602938225
exit taken: 0 with confidence: 0.7166409937163163
exit taken: 0 with confidence: 0.7072659780457483
exit taken: 0 with confidence: 0.7177791417991979
exit taken: 0 with confidence: 0.7185269034184961
exit taken: 0 with confidence: 0.7221214855182521
exit taken: 0 with confidence: 0.7089765978413601
exit taken: 0 with confidence: 0.7186967191096244
exit taken: 0 with confidence: 0.706975665336567
exit taken: 0 with confidence: 0.7019761555206072
e

Evaluating:  95%|█████████▌| 953/1000 [00:03<00:00, 343.43image/s]

exit taken: 0 with confidence: 0.717097454220112
exit taken: 0 with confidence: 0.7150562166611553
exit taken: 0 with confidence: 0.7214257774831936
exit taken: 2 with confidence: 0.7015877744445026
exit taken: 0 with confidence: 0.7155322660220467
exit taken: 2 with confidence: 0.7065247560278913
exit taken: 0 with confidence: 0.7277376850630499
exit taken: 0 with confidence: 0.7205309866382494
exit taken: 0 with confidence: 0.7203618905571271
exit taken: 0 with confidence: 0.7235249624860003
exit taken: 0 with confidence: 0.7200719027706609
exit taken: 0 with confidence: 0.721150161264646
exit taken: 0 with confidence: 0.7068290705352718
exit taken: 0 with confidence: 0.7020557025156294
exit taken: 0 with confidence: 0.7218642059855964
exit taken: 0 with confidence: 0.728548905025154
exit taken: 0 with confidence: 0.7203760838723856
exit taken: 0 with confidence: 0.7195168548812609
exit taken: 0 with confidence: 0.7142362007494107
exit taken: 0 with confidence: 0.7198921848856334
exi

Evaluating: 100%|██████████| 1000/1000 [00:03<00:00, 290.60image/s]

exit taken: 0 with confidence: 0.7197250285225215
exit taken: 0 with confidence: 0.7239832980602182
exit taken: 1 with confidence: 0.7072996034903379
exit taken: 0 with confidence: 0.7003933380604199
exit taken: 0 with confidence: 0.721026648440381
exit taken: 0 with confidence: 0.7196451449320304
exit taken: 0 with confidence: 0.7212930672384149
exit taken: 0 with confidence: 0.7049352061866598
exit taken: 1 with confidence: 0.7064711195050075
exit taken: 0 with confidence: 0.7221446525732005
Number of correct: 295, wrong: 17

Total # of Images: 312 TFLite model overall test accuracy with multi-exit: 0.9455
Exit 1: taken 283 times, average confidence = 0.7159
Exit 2: taken 11 times, average confidence = 0.7060
Exit 3: taken 18 times, average confidence = 0.7073


# Convert .tflite to C arrays

In [7]:
import binascii

def convert_to_c_array(bytes) -> str:
  hexstr = binascii.hexlify(bytes).decode("UTF-8")
  hexstr = hexstr.upper()
  array = ["0x" + hexstr[i:i + 2] for i in range(0, len(hexstr), 2)]
  array = [array[i:i+10] for i in range(0, len(array), 10)]
  return ",\n  ".join([", ".join(e) for e in array])

def save_c_array_to_file(tflite_file_path):
  tflite_binary = open(tflite_file_path, 'rb').read()
  ascii_bytes = convert_to_c_array(tflite_binary)
  c_file = "const unsigned char tf_model[] DATA_ALIGN_ATTRIBUTE = {\n  " + ascii_bytes + "\n};\nconst int tf_model_len = " + str(len(tflite_binary)) + ";"
  # print(c_file)
  print(f"c_array_len: {len(c_file)}")
  # open("model.h", "w").write(c_file)
  open(tflite_file_path.split(".")[0]+".cpp", "w").write(c_file)


In [8]:
save_c_array_to_file("final_result/k_4_c_2_model_chunk_0.tflite")
save_c_array_to_file("final_result/k_4_c_2_model_chunk_1.tflite")
# save_c_array_to_file("final_result/k_4_c_3_model_chunk_2.tflite")

c_array_len: 148493
c_array_len: 271873
